In [1]:
import requests

import re
import urllib
import math
import time
import random

import pandas as pd
import sqlite3

In [2]:
my_headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'music.163.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
def getJSON(url, headers):
    """ Get JSON from the destination URL
    @ param url: destination url, str 
    @ param headers: request headers, dict
    @ return json: result, json
    """
    res = requests.get(url, headers=headers)
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    json = res.json()
    return json

In [4]:
def countPages(total, limit):
    """ Count pages
    @ param total: total num of records, int
    @ param limit: limit per page, int
    @ return page: num of pages, int
    """
    page = math.ceil(total/limit) 
    return page

In [5]:
def getSongInfo(song_list):
    """ Parse song info
    @ param song_list: list of songs, list
    @ return song_info_list: result, list
    """
    song_info_list = []
    
    for song in song_list:
        song_info = []
    
        song_info.append(song['id'])
        song_info.append(song['name'])
    
        artists_name = ''
        artists = song['artists']
        for artist in artists:
            artists_name += artist['name'] + ','
        song_info.append(artists_name)
    
        song_info.append(song['album']['name'])
        song_info.append(song['album']['id'])
        song_info.append(song['duration'])
        
        song_info_list.append(song_info)
        
    return song_info_list

In [6]:
def getSongList(key, limit=30):
    """ Get a list of songs
    @ param key: key word, str
    @ param limit: limit per page, int, default 30
    @ return result: result, DataFrame
    """
    total_list = []
    key = urllib.parse.quote(key)
    url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=0&total=true&limit='
    
    first_page = getJSON(url, my_headers)
    song_count = first_page['result']['songCount']
    page_num = countPages(song_count, limit)
    
    for n in range(page_num):
        url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=' + str(n*limit) + '&total=true&limit=' + str(limit)
        tmp = getJSON(url, my_headers)
        song_list = getSongInfo(tmp['result']['songs'])
        total_list += song_list
        
        print('第 {0}/{1} 页爬取完成'.format(n+1, page_num))
        time.sleep(random.randint(2, 4)) 
        
    df = pd.DataFrame(data = total_list, columns=['song_id', 'song_name', 'artists', 'album_name', 'album_id', 'duration'])
    return df

In [7]:
def getComment(comments):
    """ Parse song comment
        @ param comments: list of comments, list
        @ return comments_list: result, list
    """
    comments_list = []
    
    for comment in comments:
        comment_info = []
        comment_info.append(comment['commentId'])
        comment_info.append(comment['user']['userId'])
        comment_info.append(comment['user']['nickname'])
        comment_info.append(comment['user']['avatarUrl'])
        comment_info.append(comment['content'])
        comment_info.append(comment['likedCount'])
        comments_list.append(comment_info)
        
    return comments_list

In [8]:
def getSongComment(id, limit=20):
    """ Get Song Comments
    @ param id: song id, int
    @ param limit: limit per page, int, default 20
    @ return result: result, DataFrame
    """
    total_comment = []
    url = 'http://music.163.com/api/v1/resource/comments/R_SO_4_' + str(id) +  '?limit=20&offset=0'
    
    first_page = getJSON(url, my_headers)
    total = first_page['total']
    page_num = countPages(total, limit)
    
    for n in range(page_num):
        url = 'http://music.163.com/api/v1/resource/comments/R_SO_4_' + str(id) +  '?limit=' + str(limit) + '&offset=' + str(n*limit)
        tmp = getJSON(url, my_headers)
        comment_list = getComment(tmp['comments'])
        total_comment += comment_list
        
        print('第 {0}/{1} 页爬取完成'.format(n+1, page_num))
        time.sleep(random.randint(2, 4)) 
        
    df = pd.DataFrame(data = total_comment, columns=['comment_id', 'user_id', 'user_nickname', 'user_avatar', 'content', 'likeCount'])
    df['song_id'] = str(id)
    return df

In [9]:
conn = sqlite3.connect('netease_cloud_music.db')

In [10]:
#artist='窦唯'

In [27]:
#song_df = getSongList(artist, 100)
#song_df = song_df[song_df['artists'].str.contains(artist)]
#song_df.drop_duplicates(subset=['song_id'], keep='first', inplace=True)
#song_df.to_sql(name='song', con=conn, if_exists='append', index=False)

第 1/5 页爬取完成
第 2/5 页爬取完成
第 3/5 页爬取完成
第 4/5 页爬取完成
第 5/5 页爬取完成


In [28]:
sql = '''
    SELECT song_id
    FROM song
    WHERE artists LIKE '%窦唯%'
'''

#song_id = pd.read_sql(sql, con=conn)

In [31]:
song_id.head()

,song_id
245,27483437
246,474739413
247,474739437
248,76915
249,77011


In [32]:
song_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 245 to 444
Data columns (total 1 columns):
song_id    200 non-null int64
dtypes: int64(1)
memory usage: 1.6 KB


In [33]:
comment_df = pd.DataFrame()
for index, id in zip(song_id.index, song_id['song_id']):
    print('开始爬取第 {0}/{1} 首, {2}'.format(index+1, len(song_id['song_id']), id))
    tmp_df = getSongComment(id, 100)
    comment_df = pd.concat([comment_df, tmp_df])
comment_df.drop_duplicates(subset=['comment_id'], keep='first', inplace=True)
comment_df.to_sql(name='comment', con=conn, if_exists='append', index=False)
print('已成功保存至数据库！')

开始爬取第 246/200 首, 27483437
第 1/1 页爬取完成
开始爬取第 247/200 首, 474739413
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 248/200 首, 474739437
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 249/200 首, 76915
第 1/1 页爬取完成
开始爬取第 250/200 首, 77011
第 1/6 页爬取完成
第 2/6 页爬取完成
第 3/6 页爬取完成
第 4/6 页爬取完成
第 5/6 页爬取完成
第 6/6 页爬取完成
开始爬取第 251/200 首, 77019
第 1/1 页爬取完成
开始爬取第 252/200 首, 25829547
第 1/1 页爬取完成
开始爬取第 253/200 首, 25864443
第 1/1 页爬取完成
开始爬取第 254/200 首, 25865011
第 1/1 页爬取完成
开始爬取第 255/200 首, 25896091
第 1/3 页爬取完成
第 2/3 页爬取完成
第 3/3 页爬取完成
开始爬取第 256/200 首, 26031011
第 1/4 页爬取完成
第 2/4 页爬取完成
第 3/4 页爬取完成
第 4/4 页爬取完成
开始爬取第 257/200 首, 26031019
第 1/8 页爬取完成
第 2/8 页爬取完成
第 3/8 页爬取完成
第 4/8 页爬取完成
第 5/8 页爬取完成
第 6/8 页爬取完成
第 7/8 页爬取完成
第 8/8 页爬取完成
开始爬取第 258/200 首, 26577371
第 1/1 页爬取完成
开始爬取第 259/200 首, 26577395
第 1/1 页爬取完成
开始爬取第 260/200 首, 26577427
第 1/1 页爬取完成
开始爬取第 261/200 首, 27483307
第 1/1 页爬取完成
开始爬取第 262/200 首, 27483387
第 1/1 页爬取完成
开始爬取第 263/200 首, 27483435
第 1/1 页爬取完成
开始爬取第 264/200 首, 474739411
第 1/3 页爬取完成
第 2/3 页爬取完成
第 3/3 页爬取完成
开始爬取第 265/200 首, 474739435
第 1/2

第 1/1 页爬取完成
开始爬取第 391/200 首, 5273325
第 1/1 页爬取完成
开始爬取第 392/200 首, 5274515
第 1/1 页爬取完成
开始爬取第 393/200 首, 5278646
第 1/1 页爬取完成
开始爬取第 394/200 首, 543656177
第 1/1 页爬取完成
开始爬取第 395/200 首, 5277513
第 1/1 页爬取完成
开始爬取第 396/200 首, 5273295
第 1/1 页爬取完成
开始爬取第 397/200 首, 5270630
第 1/1 页爬取完成
开始爬取第 398/200 首, 26031010
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 399/200 首, 26031018
第 1/4 页爬取完成
第 2/4 页爬取完成
第 3/4 页爬取完成
第 4/4 页爬取完成
开始爬取第 400/200 首, 26577370
第 1/1 页爬取完成
开始爬取第 401/200 首, 26577394
第 1/10 页爬取完成
第 2/10 页爬取完成
第 3/10 页爬取完成
第 4/10 页爬取完成
第 5/10 页爬取完成
第 6/10 页爬取完成
第 7/10 页爬取完成
第 8/10 页爬取完成
第 9/10 页爬取完成
第 10/10 页爬取完成
开始爬取第 402/200 首, 26577426
第 1/1 页爬取完成
开始爬取第 403/200 首, 27483306
第 1/1 页爬取完成
开始爬取第 404/200 首, 27483386
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 405/200 首, 474739410
第 1/5 页爬取完成
第 2/5 页爬取完成
第 3/5 页爬取完成
第 4/5 页爬取完成
第 5/5 页爬取完成
开始爬取第 406/200 首, 474739434
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 407/200 首, 76992
第 1/2 页爬取完成
第 2/2 页爬取完成
开始爬取第 408/200 首, 25858520
第 1/1 页爬取完成
开始爬取第 409/200 首, 25864440
第 1/1 页爬取完成
开始爬取第 410/200 首, 2587